In [1]:
import pandas as pd 
import csv
from builtins import open, bytes
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
nltk.download('stopwords')
from deep_translator import GoogleTranslator
from langdetect import detect
import seaborn as sns
from recordlinkage.standardise import clean
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean_and_translate(bp):
    # detect the language of each comments
    from langdetect import detect
    def detect_text(text):
        try:
            return detect(text)
        except: 
            return 'Unknown'

    # create translation function
    from deep_translator import GoogleTranslator
    def translate(text): 
        translator = GoogleTranslator(source = 'auto', target='english')
        translation = translator.translate(text)
        return translation 

    # remove stop words function 
    def remove_stop_words(text): 
        final_text = []
        text = text.lower()
        stop = set(stopwords.words('english'))
        for i in text.split():
            if i.strip() not in stop and i.strip() != 'https':
                final_text.append(i.strip())
        return " ".join(final_text)
    
    # clean comments 
    bp['full_text'] = clean(bp['full_text'])

    # remove stop words from each comments 
    bp['full_text'] = bp['full_text'].apply(remove_stop_words)

    # change the data type of the comment 
    bp = bp.astype({'full_text': 'string'})

    # remove http links 
    bp['full_text'] = bp['full_text'].str.replace(r'http\S+', '', regex=True)

    # delete rows with less than 5 character in comments
    bp['length'] = bp['full_text'].str.len()
    drop_lst = bp[bp['length'] < 5].index.to_list()
    bp = bp.drop(drop_lst, axis = 0)
    bp = bp.reset_index()
    
    # create a language column for comment language identifier and make index list for it 
    # bp['language'] = bp['full_text'].apply(detect_text)
    # index_lst = bp[bp['language'] != 'en'].index.to_list()

    # create the translated column
    bp['translated'] = bp['full_text']

    # # apply translation 
    # err_df = bp.iloc[index_lst]
    # err_df['translated'] = err_df['full_text'].apply(translate)

    # update the final data 
    bp = bp.drop(columns='index')
    # bp = bp.drop(columns='Unnamed: 0')

    return bp 

In [3]:
from langdetect import detect
def detect_text(text):
    try:
        return detect(text)
    except: 
        return 'Unknown'

def translate(text): 
    translator = GoogleTranslator(source = 'auto', target='english')
    translation = translator.translate(text)
    return translation 

## Iran

In [151]:
ir = pd.read_csv('/Users/chris/Desktop/Thesis/public_opinion_thesis/Data/iran_cleaned.csv') # read in the data from csv file 
ir.drop(columns=['Unnamed: 0','truncated_full_text'],inplace=True) # drop unnecessary column
ir['full_text'] = ir['full_text'].astype(str)   # transfer text column into string type 

In [194]:
# pre-processing data by removing stop words and etc. 
ir_cleaned = clean_and_translate(ir)
ir_cleaned.head()

,url,full_text,user_id_str,month,day,year,length,translated
0,https://twitter.com/DemocraticIran3/status/991...,politicalshort although nothing empty threats ...,949663637051932672,Apr,30,2018,105,politicalshort although nothing empty threats ...
1,https://twitter.com/trex9123/status/9911048901...,glennthrush arifleischer netanyahu sec mattis ...,464095397,Apr,30,2018,106,glennthrush arifleischer netanyahu sec mattis ...
2,https://twitter.com/Jackhow74060123/status/991...,anncoulter realdonaldtrump iran biggest threat...,965799103065026560,Apr,30,2018,225,anncoulter realdonaldtrump iran biggest threat...
3,https://twitter.com/850Sunny/status/9911048702...,obamas hidden iran deal giveaway via politico,1887844819,Apr,30,2018,46,obamas hidden iran deal giveaway via politico
4,https://twitter.com/marygnip/status/9911048700...,iran adhering obama agreement would possible,1282228513,Apr,30,2018,45,iran adhering obama agreement would possible


In [ ]:
# identify language of each comments 
ir_cleaned['language'] = ir_cleaned['full_text'].apply(detect_text)
# generating list of comments that are not in English 
index_lst = ir_cleaned[ir_cleaned['language'] != 'en'].index.to_list()
# creating sub_dataframe to store tweets that are not written in English 
ir_need_trans = ir_cleaned.iloc[index_lst]

In [196]:
# manually divide the dataframe into several parts
ir_df1 = ir_need_trans[:1000]
ir_df2 = ir_need_trans[1000:2000]
ir_df3 = ir_need_trans[2000:3000]
ir_df4 = ir_need_trans[3000:4000]
ir_df5 = ir_need_trans[4000:]

In [198]:
# implement the translation formula to translate tweets text into English
ir_df1['translated'] = ir_df1['full_text'].apply(translate)
ir_df2['translated'] = ir_df2['full_text'].apply(translate)
ir_df3['translated'] = ir_df3['full_text'].apply(translate)
ir_df4['translated'] = ir_df4['full_text'].apply(translate)
ir_df5['translated'] = ir_df5['full_text'].apply(translate)

/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/2876833339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir_df1['translated'] = ir_df1['full_text'].apply(translate)
/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/2876833339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir_df2['translated'] = ir_df2['full_text'].apply(translate)
/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/2876833339.py:3: SettingWithCopyWarning: 
A value is trying to be set o

In [199]:
translated_seg = pd.concat([ir_df1, ir_df2, ir_df3, ir_df4, ir_df5], ignore_index=True)

In [201]:
ir_cleaned

,url,full_text,user_id_str,month,day,year,length,translated,language
0,https://twitter.com/DemocraticIran3/status/991...,politicalshort although nothing empty threats ...,949663637051932672,Apr,30,2018,105,politicalshort although nothing empty threats ...,en
1,https://twitter.com/trex9123/status/9911048901...,glennthrush arifleischer netanyahu sec mattis ...,464095397,Apr,30,2018,106,glennthrush arifleischer netanyahu sec mattis ...,en
2,https://twitter.com/Jackhow74060123/status/991...,anncoulter realdonaldtrump iran biggest threat...,965799103065026560,Apr,30,2018,225,anncoulter realdonaldtrump iran biggest threat...,en
3,https://twitter.com/850Sunny/status/9911048702...,obamas hidden iran deal giveaway via politico,1887844819,Apr,30,2018,46,obamas hidden iran deal giveaway via politico,es
4,https://twitter.com/marygnip/status/9911048700...,iran adhering obama agreement would possible,1282228513,Apr,30,2018,45,iran adhering obama agreement would possible,en
...,...,...,...,...,...,...,...,...,...
10395,https://twitter.com/therealck73/status/1708265...,gangaram012 alokdubey1408 stats feed missed ir...,1498343627935858695,Sep,30,2023,73,gangaram012 alokdubey1408 stats feed missed ir...,en
10396,https://twitter.com/xTatsuyaLing/status/170826...,solo algunos pensamientos que se van rpido cre...,1170625199580110848,Sep,30,2023,85,solo algunos pensamientos que se van rpido cre...,es
10397,https://twitter.com/azadi_ye_Iran/status/17082...,alef20000,1632234376544112640,Sep,30,2023,9,alef20000,cy
10398,https://twitter.com/johnwindt01/status/1708264...,timothydsnyder mention iranwhich building nucl...,1519339462471438337,Sep,30,2023,97,timothydsnyder mention iranwhich building nucl...,en


In [202]:
ir_cleaned.update(translated_seg)
ir_cleaned = ir_cleaned.rename(columns={'full_text':'orig_text'})
ir_cleaned = ir_cleaned.drop(columns='length')
ir_cleaned = ir_cleaned.drop(columns='language')
ir_cleaned['country'] = 'iran'

In [208]:
ir_cleaned.to_csv('/Users/chris/Desktop/Thesis/Data/iran_translated.csv')

## China

In [214]:
china = pd.read_csv('/Users/chris/Desktop/Thesis/Data/china_cleaned.csv') # read in the data from csv file 
china.drop(columns=['Unnamed: 0'],inplace=True) # drop unnecessary column
china.head() # transfer text column into string type 

,full_text,url,user_id_str,month,day,year
0,@BeautyNDFeast @BeautyNDFeast Delicious! Did t...,https://twitter.com/China_Spice/status/9911049...,1315115694,Apr,30,2018
1,Listen to White china mp3 by DJ CandyApple #np...,https://twitter.com/DjCandyApple/status/991104...,824988053345144832,Apr,30,2018
2,Places I want houses in the future.\n1. West F...,https://twitter.com/McChigga/status/9911048452...,363787440,Apr,30,2018
3,"kids ready to target @ Gucun, Shanghai, China ...",https://twitter.com/AlexSu007/status/991104836...,924952614852222977,Apr,30,2018
4,ESL Teaching job China - 5 ESL Teachers https:...,https://twitter.com/eslteacherboard/status/991...,973726399348396032,Apr,30,2018


In [215]:
# pre-processing data by removing stop words and etc. 
china_cleaned = clean_and_translate(china)
china_cleaned.head()

,full_text,url,user_id_str,month,day,year,length,translated
0,beautyndfeast beautyndfeast delicious paneer g...,https://twitter.com/China_Spice/status/9911049...,1315115694,Apr,30,2018,73,beautyndfeast beautyndfeast delicious paneer g...
1,listen white china mp3 dj candyapple np soundc...,https://twitter.com/DjCandyApple/status/991104...,824988053345144832,Apr,30,2018,51,listen white china mp3 dj candyapple np soundc...
2,places want houses future1 west falmouth 2 cle...,https://twitter.com/McChigga/status/9911048452...,363787440,Apr,30,2018,134,places want houses future1 west falmouth 2 cle...
3,kids ready target gucun shanghai china,https://twitter.com/AlexSu007/status/991104836...,924952614852222977,Apr,30,2018,39,kids ready target gucun shanghai china
4,esl teaching job china 5 esl teachers,https://twitter.com/eslteacherboard/status/991...,973726399348396032,Apr,30,2018,38,esl teaching job china 5 esl teachers


In [216]:
# identify language of each comments 
china_cleaned['language'] = china_cleaned['full_text'].apply(detect_text)
# generating list of comments that are not in English 
index_lst = china_cleaned[china_cleaned['language'] != 'en'].index.to_list()
# creating sub_dataframe to store tweets that are not written in English 
china_need_trans = china_cleaned.iloc[index_lst]

In [217]:
china_need_trans.count()

full_text      2786
url            2786
user_id_str    2786
month          2786
day            2786
year           2786
length         2786
translated     2786
language       2786
dtype: int64

In [219]:
# manually divide the dataframe into several parts
china_df1 = china_need_trans[:700]
china_df2 = china_need_trans[700:1400]
china_df3 = china_need_trans[1400:2100]
china_df4 = china_need_trans[2100:]

In [220]:
# implement the translation formula to translate tweets text into English
china_df1['translated'] = china_df1['full_text'].apply(translate)
china_df2['translated'] = china_df2['full_text'].apply(translate)
china_df3['translated'] = china_df3['full_text'].apply(translate)
china_df4['translated'] = china_df4['full_text'].apply(translate)

/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/1277191382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  china_df1['translated'] = china_df1['full_text'].apply(translate)
/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/1277191382.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  china_df2['translated'] = china_df2['full_text'].apply(translate)
/var/folders/nq/5z_sx69s5ngg5p145v4g54t80000gn/T/ipykernel_41092/1277191382.py:4: SettingWithCopyWarning: 
A value is trying

In [221]:
translated_seg = pd.concat([china_df1, china_df2, china_df3, china_df4], ignore_index=True)
china_cleaned.update(translated_seg)
china_cleaned = china_cleaned.rename(columns={'full_text':'orig_text'})
china_cleaned = china_cleaned.drop(columns='length')
china_cleaned = china_cleaned.drop(columns='language')
china_cleaned['country'] = 'china'

In [222]:
china_cleaned

,orig_text,url,user_id_str,month,day,year,translated,country
0,pmoindia narendramodi turned tables china good...,https://twitter.com/Jaya462/status/99110477607...,2.898726e+08,Apr,30.0,2018.0,pmoindia narendramodi turned tables china good...,china
1,jwalin china,https://twitter.com/iamkar4n/status/9911047529...,2.394454e+09,Apr,30.0,2018.0,jwalin china,china
2,quero ir pra china pega um zoin kkkk,https://twitter.com/yasfonceca/status/99110469...,8.805590e+17,Apr,30.0,2018.0,"I want to go to China, get a zoin hahaha",china
3,el prximo paso para la poltica econmica de china,https://twitter.com/flglobalconsult/status/991...,7.547805e+17,Apr,30.0,2018.0,The next step for China's economic policy,china
4,cheguei um ponto que um dia antes da minha dat...,https://twitter.com/edzkz/status/9911044972534...,7.427949e+17,Apr,30.0,2018.0,I reached a point where one day before my due ...,china
...,...,...,...,...,...,...,...,...
11248,hackers linked chinese government stole around...,https://twitter.com/AdamMilstein/status/170749...,9.363778e+07,Sep,28.0,2023.0,hackers linked chinese government stole around...,china
11249,clear channel pay 26 million settle sec allega...,https://twitter.com/business/status/1707519906...,3.471336e+07,Sep,28.0,2023.0,clear channel pay 26 million settle sec allega...,china
11250,chinese government developed increasingly stat...,https://twitter.com/ForeignAffairs/status/1708...,2.111466e+07,Sep,30.0,2023.0,chinese government developed increasingly stat...,china
11251,us state dept spokesman seen increase tempo co...,https://twitter.com/JChengWSJ/status/170722888...,3.827884e+08,Sep,28.0,2023.0,us state dept spokesman seen increase tempo co...,china


In [223]:
china_cleaned.to_csv('/Users/chris/Desktop/Thesis/Data/china_translated.csv')

## Russia

In [12]:
ru = pd.read_csv('/Users/chris/Desktop/Thesis/public_opinion_thesis/Data/russia_cleaned.csv') # read in the data from csv file 
# ru.drop(columns=['Unnamed: 0','truncated_full_text'],inplace=True) # drop unnecessary column
ru['full_text'] = ru['full_text'].astype(str)   # transfer text column into string type 

In [13]:
# pre-processing data by removing stop words and etc. 
ru_cleaned = clean_and_translate(ru)
ru_cleaned.head()

,Unnamed: 0,url,full_text,user_id_str,ulr,month,day,year,length,translated
0,6401,https://twitter.com/elizadimitria/status/99107...,queen russia doesnt get half hype deserves hf ...,1504991659,NaN,Apr,30,2018,73,queen russia doesnt get half hype deserves hf ...
1,6402,https://twitter.com/mac123_m/status/9910858686...,theresas tories grenfell tower disaster windru...,2288515068,NaN,Apr,30,2018,245,theresas tories grenfell tower disaster windru...
2,6403,https://twitter.com/keithboykin/status/9910951...,100 criminal counts 22 indictments 5 guilty pl...,21728303,NaN,Apr,30,2018,178,100 criminal counts 22 indictments 5 guilty pl...
3,6404,https://twitter.com/usacsmret/status/991099321...,levin trump russia conspiracy coup attempt goe...,1078094414,NaN,Apr,30,2018,70,levin trump russia conspiracy coup attempt goe...
4,6405,https://twitter.com/Thomas1774Paine/status/991...,james comey calls house russia probe wreck dis...,275276082,NaN,Apr,30,2018,79,james comey calls house russia probe wreck dis...


In [14]:
# identify language of each comments 
ru_cleaned['language'] = ru_cleaned['full_text'].apply(detect_text)
# generating list of comments that are not in English 
index_lst = ru_cleaned[ru_cleaned['language'] != 'en'].index.to_list()
# creating sub_dataframe to store tweets that are not written in English 
ru_need_trans = ru_cleaned.iloc[index_lst]

In [15]:
# manually divide the dataframe into several parts
ru_df1 = ru_need_trans[:1000]
ru_df2 = ru_need_trans[1000:2000]
ru_df3 = ru_need_trans[2000:3000]
ru_df4 = ru_need_trans[3000:4000]
ru_df5 = ru_need_trans[4000:]

In [16]:
# implement the translation formula to translate tweets text into English
ru_df1['translated'] = ru_df1['full_text'].apply(translate)
ru_df2['translated'] = ru_df2['full_text'].apply(translate)
ru_df3['translated'] = ru_df3['full_text'].apply(translate)
ru_df4['translated'] = ru_df4['full_text'].apply(translate)
ru_df5['translated'] = ru_df5['full_text'].apply(translate)

In [17]:
translated_seg = pd.concat([ru_df1, ru_df2, ru_df3, ru_df4, ru_df1], ignore_index=True)

In [18]:
ru_cleaned.update(translated_seg)
ru_cleaned = ru_cleaned.rename(columns={'full_text':'orig_text'})
ru_cleaned = ru_cleaned.drop(columns='length')
ru_cleaned = ru_cleaned.drop(columns='language')
ru_cleaned['country'] = 'russia'

In [19]:
ru_cleaned.to_csv('/Users/chris/Desktop/Thesis/public_opinion_thesis/Data/russia_translated.csv')

## North Korean

In [4]:
nk = pd.read_csv('/Users/chris/Desktop/Thesis/public_opinion_thesis/Data/northkorean_cleaned.csv') # read in the data from csv file 
nk['full_text'] = nk['full_text'].astype(str)   # transfer text column into string type 

In [5]:
# pre-processing data by removing stop words and etc. 
nk_cleaned = clean_and_translate(nk)
nk_cleaned.head()

,Unnamed: 0,url,full_text,user_id_str,month,day,year,length,translated
0,9688,https://twitter.com/MightyBusterBro/status/991...,moronic meme isbased facts solid thosebacking ...,3949049181,Apr,30,2018,210,moronic meme isbased facts solid thosebacking ...
1,9689,https://twitter.com/LeeCamp/status/99103191188...,north koreans want peace south koreans want pe...,19580890,Apr,30,2018,192,north koreans want peace south koreans want pe...
2,9690,https://twitter.com/ComedyCentral/status/99098...,meet south korean whose ball roof north korean...,23827692,Apr,30,2018,61,meet south korean whose ball roof north korean...
3,9691,https://twitter.com/ConservativeTht/status/990...,south korean president moon jae reportedly sai...,719276823809540096,Apr,30,2018,181,south korean president moon jae reportedly sai...
4,9692,https://twitter.com/FoxBusiness/status/9910971...,south korean president moon jae said monday pr...,56413858,Apr,30,2018,166,south korean president moon jae said monday pr...


In [6]:
# identify language of each comments 
nk_cleaned['language'] = nk_cleaned['full_text'].apply(detect_text)
# generating list of comments that are not in English 
index_lst = nk_cleaned[nk_cleaned['language'] != 'en'].index.to_list()
# creating sub_dataframe to store tweets that are not written in English 
nk_need_trans = nk_cleaned.iloc[index_lst]

In [7]:
len(index_lst)

244

In [8]:
# manually divide the dataframe into several parts
nk_df1 = nk_need_trans[:1000]

In [9]:
# implement the translation formula to translate tweets text into English
nk_df1['translated'] = nk_df1['full_text'].apply(translate)

In [10]:
nk_cleaned.update(nk_df1)
nk_cleaned = nk_cleaned.rename(columns={'full_text':'orig_text'})
nk_cleaned = nk_cleaned.drop(columns='length')
nk_cleaned = nk_cleaned.drop(columns='language')
nk_cleaned['country'] = 'nk'

In [11]:
nk_cleaned.to_csv('/Users/chris/Desktop/Thesis/public_opinion_thesis/Data/nk_translated.csv')